In [101]:
import os
import re
from dotenv import load_dotenv
from groq import Groq
import sqlalchemy
print("Libraries loaded and environment variables are ready.")


Libraries loaded and environment variables are ready.


In [108]:
db_url = os.getenv("DB_URL")

try:
    # Create a SQLAlchemy engine to connect to the database
    engine = sqlalchemy.create_engine(db_url)
    
    # Test the connection by fetching the first customer
    with engine.connect() as connection:
        result = connection.execute(sqlalchemy.text("SELECT name FROM customers LIMIT 1;"))
        for row in result:
            print(f"Successfully connected to the database. Found customer: {row.name}")

except Exception as e:
    print(f"Failed to connect to the database and Error: {e}")


Successfully connected to the database. Found customer: Aisha Khan


In [109]:
# Initialize the Groq client with your API key
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# The updated system prompt for the simple, single-table schema
SYSTEM_PROMPT = """You are a specialized SQL Code Bot. Your single purpose is to convert a user's question into a single, clean, valid PostgreSQL query for the table provided below.

### PRIMARY DIRECTIVE
You will output ONLY the SQL query required. Nothing else. No comments, no explanations.

### DATABASE SCHEMA
CREATE TABLE customers (
    customer_id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    gender VARCHAR(50),
    location VARCHAR(255)
);

### RULES
1.  For all text comparisons (gender, location, name), you MUST use the `ILIKE` operator for case-insensitivity.
2.  For names, use wildcards for partial matches. Example: `name ILIKE '%arjun%'`.
3.  Do NOT use JOINs. All data is in the `customers` table.

### EXAMPLES
* **User:** "Show me all female customers from Mumbai"
* **Your SQL:** SELECT * FROM customers WHERE gender ILIKE 'female' AND location ILIKE 'mumbai';

* **User:** "who is arjun"
* **Your SQL:** SELECT * FROM customers WHERE name ILIKE '%arjun%';

### FINAL OUTPUT
Your entire response must be only the raw SQL query.
"""

print("System prompt for the simple schema is ready.")

System prompt for the simple schema is ready.


In [ ]:
# Prompt the user to enter their question
user_query = input("Please enter your query: ")

print(f"\nUser Query: '{user_query}'\n")

# Send the user query and system prompt to the LLM
try:
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": SYSTEM_PROMPT,
            },
            {
                "role": "user",
                "content": user_query,
            }
        ],
        model="llama3-8b-8192",
    )

    # Extract the potentially messy response from the LLM
    raw_response = chat_completion.choices[0].message.content.strip()
    
    # Use regex to find and extract only the SQL query from the response
    sql_match = re.search(r"SELECT.*?;", raw_response, re.DOTALL | re.IGNORECASE)
    
    if not sql_match:
        raise ValueError(f"The LLM did not return a valid SQL query. Response was: {raw_response}")

    generated_sql = sql_match.group(0)
    print(f"Generated SQL: \n{generated_sql}\n")

    # Execute the extracted SQL query against your database
    with engine.connect() as connection:
        result = connection.execute(sqlalchemy.text(generated_sql))
        # Get column names from the result proxy to create a list of dictionaries
        columns = result.keys()
        rows = [dict(zip(columns, row)) for row in result.fetchall()]
        
        print("Query Results:")
        if rows:
            # A simple way to pretty-print the results
            for row in rows:
                print(row)
        else:
            print("No results found.")

except Exception as e:
    print(f"An error occurred: {e}")
